# 基于深度学习的自动标定算法

## CalibNet
### 一、介绍
CalibNet[1]是有监督的深度学习自动标定算法(KITTI数据集)，但监督信号不是外参而是目标点云(因此文献说不是显式监督)。CalibNet输入点云、图像、相机内参，输出外参。算法流程图如下所示。

<p align="center"><img src="../resources/paper-nn/calibnet-workflow.png" width=50%></p>
<h6 align="center">CalibNet工作流程图</h6>

具体算法流程如下所示。
1. 将输入点云投影至图像平面，得到初始稀疏深度图。稀疏深度图和可见光图像作为神经网络输入。
2. 对深度图采用$5 \times 5$的max pooling得到较稠密深度图。
3. 使用如下网络架构回归出平移向量$t \in \mathbb{R}^3$，旋转向量$w \in so(3)$
4. 旋转向量经指数映射(罗德里格斯公式)$R = I + \frac{\hat{w}}{||w||}sin(||w||) + \frac{\hat{w}^2}{||w||^2}(1-cos(||w||))$得到旋转向量，然后将初始深度图重投影到新外参下。
5. 构造新深度图与目标深度图的光度损失(photometric loss)和点云损失(pointcloud loss)，进行训练。


<p align="center"><img src="../resources/paper-nn/calibnet-nn.png" width=50%></p>
<h6 align="center">CalibNet网络架构</h6>


### 二、损失
1. 光度损失(photometric loss):其中$D_{init}$是初始深度图，$D_{gt}$是真值深度图。
$$L_{photo}=(D_{gt} - KT\pi^{-1}D_{init})^2$$
2. 点云距离损失(Pointcloud distance loss):这一部分使用了包括Chamfer Distance，Earth Mover's Distance，Centroid ICP Distance等。

### 三、结论
CalibNet解决了$\pm20°$旋转，$\pm 0.2m$平移，获得了MAE为0.41°和4.34cm的误差。CalibNet网络的估计可以作为其它基于优化方法的初始值，这样可以进一步提升外参估计精度。

## 参考资料
[1] Iyer, G., KarnikRam, R., Jatavallabhula, K., & Krishna, K.M. (2018). CalibNet: Geometrically Supervised Extrinsic Calibration using 3D Spatial Transformer Networks. 2018 IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS), 1110-1117.[paper](https://www.semanticscholar.org/paper/CalibNet%3A-Geometrically-Supervised-Extrinsic-using-Iyer-KarnikRam/a67e66e437331cf7dbb02d2a7296e9c32dba67d5) [code](https://github.com/epiception/CalibNet) [project](https://epiception.github.io/CalibNet/)